In [1]:
import pandas as pd
from datetime import datetime, timedelta

test_data_split = 230  #this will be the number of weeks in the test data set, remainder of data in train data set
#Calculate the split date to use
split_date = datetime.now() - timedelta(weeks=test_data_split) 
print('Split Date: {0}'.format(split_date))

Split Date: 2015-11-24 20:03:21.580775


In [2]:
#Read in the transportation data, monthly seasonally adjusted
tsi_data = pd.read_excel(".\data\Input_SeasonalData_TSI.xlsx", header=2)

#look at the data types that were inferred by Pandas during import.
tsi_data.dtypes

OBS_DATE                   datetime64[ns]
RAIL_FRT_CARLOADS_D11               int64
RAIL_FRT_INTERMODAL_D11             int64
WATERBORNE_D11                    float64
TRUCK_D11                         float64
AIR_RTMFM_D11                       int64
TSI                               float64
dtype: object

In [3]:
#Return a listing of the data
tsi_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6


In [4]:
#Read in the real gdp growth rates,seasonally adjusted, quarterly data with quarterly growth rates
gdp_data = pd.read_excel(".\data\Input_GDP_st_louis_fed_quarterly_change.xlsx")

#look at the data types that were inferred by Pandas during import.
gdp_data.dtypes

observation_date       datetime64[ns]
Real_gdp_qtr_growth           float64
dtype: object

In [5]:
#Disply a listing of the data
gdp_data.head()

,observation_date,Real_gdp_qtr_growth
0,2000-01-01,1.5
1,2000-04-01,7.5
2,2000-07-01,0.5
3,2000-10-01,2.5
4,2001-01-01,-1.1


In [6]:
#Merge the 2 datasets together based on the date. 
merged_data = pd.merge(tsi_data, gdp_data, how='left', left_on='OBS_DATE', right_on='observation_date')

merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,observation_date,Real_gdp_qtr_growth
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,2000-01-01,1.5
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,NaT,NaN
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,NaT,NaN
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,2000-04-01,7.5
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,NaT,NaN


In [7]:
#We now have 2 date columns.  Drop the date column from the gdp_data dataframe.
merged_data = merged_data.drop(['observation_date'], axis=1) #pandas can drop rows or columns, axis=1 indicates columns

In [8]:
#This now shows the dataframe without the duplicate date column observation_date.
merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,1.5
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,NaN
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,NaN
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,7.5
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,NaN


In [9]:
#Look at the counts for the data values that we have for each column.  We see the gdp data with a smaller number since
#it is quarterly and the other data is monthly.
merged_data.count()

OBS_DATE                   241
RAIL_FRT_CARLOADS_D11      241
RAIL_FRT_INTERMODAL_D11    241
WATERBORNE_D11             241
TRUCK_D11                  241
AIR_RTMFM_D11              241
TSI                        241
Real_gdp_qtr_growth         80
dtype: int64

In [10]:
#Forward fill the gdp data so that a quarterly gdp value will be used for 3 rows, with each row being a month.
#limit it only fill in 2 consective missing values.
merged_data['Real_gdp_qtr_growth'] = merged_data['Real_gdp_qtr_growth'].fillna(method='ffill', limit=2)

merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,1.5
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,1.5
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,1.5
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,7.5
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,7.5


In [11]:
#Look to see if we still have missing gdp values as that data gets released later than the transportation data.
#This will allow us to inspect any bad rows.
merged_data.loc[merged_data['Real_gdp_qtr_growth'].isnull()]

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth
240,2020-01-01,1054432,1109662,50.4,116.6,3617962,136.9,NaN


In [12]:
#drop rows where we don't have a gdp value
merged_data = merged_data.dropna(subset=['Real_gdp_qtr_growth'])

In [13]:
#Verify that all rows have gdp values 
merged_data.loc[merged_data['Real_gdp_qtr_growth'].isnull()]

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth


In [14]:
#We should now have clean data.
merged_data.count()

OBS_DATE                   240
RAIL_FRT_CARLOADS_D11      240
RAIL_FRT_INTERMODAL_D11    240
WATERBORNE_D11             240
TRUCK_D11                  240
AIR_RTMFM_D11              240
TSI                        240
Real_gdp_qtr_growth        240
dtype: int64

In [15]:
#Create a boolean column to indicate if gdp growth was positive or negative.
merged_data['gdp_is_increasing'] = merged_data.apply(lambda x: True if x['Real_gdp_qtr_growth'] > 0.0 else False, axis=1)

In [16]:
merged_data.count()

OBS_DATE                   240
RAIL_FRT_CARLOADS_D11      240
RAIL_FRT_INTERMODAL_D11    240
WATERBORNE_D11             240
TRUCK_D11                  240
AIR_RTMFM_D11              240
TSI                        240
Real_gdp_qtr_growth        240
gdp_is_increasing          240
dtype: int64

In [17]:
#look at the new column
merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth,gdp_is_increasing
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,1.5,True
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,1.5,True
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,1.5,True
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,7.5,True
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,7.5,True


In [18]:
#Look at rows where gdp growth was negative
merged_data.loc[merged_data['Real_gdp_qtr_growth'] <= 0.0]

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth,gdp_is_increasing
12,2001-01-01,1398488,759514,46.0,74.8,2553409,99.5,-1.1,False
13,2001-02-01,1399176,743794,47.6,75.3,2547226,99.9,-1.1,False
14,2001-03-01,1416765,754615,47.0,74.2,2479860,99.1,-1.1,False
18,2001-07-01,1353431,731670,47.6,74.1,2246636,97.6,-1.7,False
19,2001-08-01,1374005,745111,49.3,75.2,2318149,99.2,-1.7,False
20,2001-09-01,1376994,754272,49.0,74.4,2047594,98.1,-1.7,False
96,2008-01-01,1416854,985481,45.1,87.5,3366557,113.4,-2.3,False
97,2008-02-01,1423289,998459,43.1,85.8,3289484,111.5,-2.3,False
98,2008-03-01,1402813,971721,36.7,85.8,3249574,109.8,-2.3,False
102,2008-07-01,1427847,986893,42.5,86.2,3126320,111.1,-2.1,False


In [19]:
merged_data.head()

,OBS_DATE,RAIL_FRT_CARLOADS_D11,RAIL_FRT_INTERMODAL_D11,WATERBORNE_D11,TRUCK_D11,AIR_RTMFM_D11,TSI,Real_gdp_qtr_growth,gdp_is_increasing
0,2000-01-01,1422442,764756,55.4,80.3,2466950,105.3,1.5,True
1,2000-02-01,1425882,767958,48.6,79.8,2521852,104.4,1.5,True
2,2000-03-01,1411458,763858,52.5,74.1,2489787,99.2,1.5,True
3,2000-04-01,1400311,764144,50.8,72.8,2557332,98.1,7.5,True
4,2000-05-01,1405169,763843,52.5,73.0,2527821,98.6,7.5,True


In [20]:
merged_data['RAIL_FRT_PCT'] = merged_data['RAIL_FRT_CARLOADS_D11'].pct_change()
merged_data['RAIL_INTERMOD_PCT'] = merged_data['RAIL_FRT_INTERMODAL_D11'].pct_change()
merged_data['WATERBORNE_PCT'] = merged_data['WATERBORNE_D11'].pct_change()
merged_data['TRUCK_PCT'] = merged_data['TRUCK_D11'].pct_change()
merged_data['AIR_RTMFM_PCT'] = merged_data['AIR_RTMFM_D11'].pct_change()
merged_data['TSI_PCT'] = merged_data['TSI'].pct_change()

In [21]:
#create x_train and y_train dataframes
x_train = merged_data.loc[merged_data['OBS_DATE'] < split_date, 'RAIL_FRT_PCT':'TSI_PCT']
x_train.drop([0], inplace=True) #drop first row as it will have NA for pct_change values
y_train_class = merged_data.loc[merged_data['OBS_DATE'] < split_date, 'gdp_is_increasing'] #ydata for classifier models
y_train_class.drop([0], inplace=True) #drop first row as it will have NA for pct_change values
y_train_rate = merged_data.loc[merged_data['OBS_DATE'] < split_date, 'Real_gdp_qtr_growth'] #y data for regression models
y_train_rate.drop([0], inplace=True) #drop first row as it will have NA for pct_change values

print('x_train row count: {0}'.format(len(x_train.index)))
print('y_train_class row count: {0}'.format(len(y_train_class.index)))
print('y_train_rate row count: {0}'.format(len(y_train_rate.index)))

x_test = merged_data.loc[merged_data['OBS_DATE'] >= split_date, 'RAIL_FRT_PCT':'TSI_PCT']
y_test_class = merged_data.loc[merged_data['OBS_DATE'] >= split_date, 'gdp_is_increasing']
y_test_rate = merged_data.loc[merged_data['OBS_DATE'] >= split_date, 'Real_gdp_qtr_growth']

print('x_test row count: {0}'.format(len(x_test.index)))
print('y_test_class row count: {0}'.format(len(y_test_class.index)))
print('y_test_rate row count: {0}'.format(len(y_test_rate.index)))

x_train row count: 190
y_train_class row count: 190
y_train_rate row count: 190
x_test row count: 49
y_test_class row count: 49
y_test_rate row count: 49


In [22]:
#x_train

In [23]:
from sklearn import preprocessing

mm_scaler = preprocessing.MinMaxScaler()
x_train_minmax = mm_scaler.fit_transform(x_train)

x_test_minmax = mm_scaler.transform(x_test)

In [41]:
x_train_minmax

array([[0.61023971, 0.36376696, 0.08374721, 0.58489814, 0.35365346,
        0.51459288],
       [0.51215726, 0.32676382, 0.74398354, 0.        , 0.26465934,
        0.00842304],
       [0.52951613, 0.34895706, 0.37765618, 0.48337637, 0.36605674,
        0.48341285],
       ...,
       [0.54098784, 0.31317844, 0.49636169, 0.62299122, 0.36750227,
        0.61944235],
       [0.55462364, 0.28047808, 0.33634554, 0.63185535, 0.29529109,
        0.57961317],
       [0.22206735, 0.30504011, 0.573713  , 0.55167263, 0.27356544,
        0.43962722]])

In [25]:
#x_test_minmax

In [26]:
y_train_class.head()

1    True
2    True
3    True
4    True
5    True
Name: gdp_is_increasing, dtype: bool

In [27]:
from sklearn.linear_model import LogisticRegression
import numpy as np

#load the model
clf = LogisticRegression(solver='liblinear')
#fit the model
clf.fit(x_train_minmax, y_train_class)

y_hat_train_class = clf.predict(x_train_minmax)
print('Classifier training accuracy: {0}'.format(np.average(y_hat_train_class == y_train_class)))

#evaluate the model by using a test set
y_hat_test_class = clf.predict(x_test_minmax)
#print the accuracy
print('Classifier test accuracy: {0}'.format(np.average(y_hat_test_class == y_test_class)))

print('Done')

Training accuracy: 0.8421052631578947
Test accuracy: 1.0
Done


In [28]:
#y_hat_class

In [29]:
#x_test

In [30]:
#y_test

In [31]:
#merged_data

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

lin = LinearRegression() 

reg = lin.fit(x_train_minmax, y_train_rate)
r2coeffofdet = lin.score(x_train_minmax, y_train_rate)

print('R2 score:', r2coeffofdet)   # This is the coefficient of determination.

y_hat_train_rate = reg.predict(x_train_minmax)

RMSE = np.sqrt(mean_squared_error(y_train_rate, y_hat_train_rate))
print('RMSE:', RMSE)

print('Coefficients:', reg.coef_)
print('Intercept:', reg.intercept_)

R2 score: 0.11512538137391572
RMSE: 2.382026735271547
Coefficients: [-0.89614828  6.47008683 -0.81939544 -1.09095299  4.84888023  3.66582935]
Intercept: -2.4745189819395708


In [47]:
from sklearn.preprocessing import PolynomialFeatures 

poly = PolynomialFeatures(degree = 2) 
x_train_minmax_poly = poly.fit_transform(x_train_minmax)

poly.fit(x_train_minmax_poly, y_train_rate)

lin_poly = LinearRegression()

reg_poly = lin_poly.fit(x_train_minmax_poly, y_train_rate)

r2coeffofdet = lin_poly.score(x_train_minmax_poly, y_train_rate)
print('R2 score:', r2coeffofdet)   # This is the coefficient of determination.

y_hat_train_rate_poly = reg_poly.predict(x_train_minmax_poly)

RMSE = np.sqrt(mean_squared_error(y_train_rate, y_hat_train_rate_poly))
print('RMSE:', RMSE)

print('Coefficients:', reg_poly.coef_)
print('Intercept:', reg_poly.intercept_)

R2 score: -0.22232429364082365
RMSE: 2.799619335026216
Coefficients: [-8.52230983e+14  1.14718868e+01 -1.68857342e+01  6.52899388e+01
  2.17374533e+02  3.64851068e+01 -2.35036886e+02  1.39417277e+01
 -8.64791357e+00 -2.71734521e+01 -2.00966173e+01 -3.10953483e+01
  2.09461382e+01 -1.56650553e+01  2.71388537e+01  8.44790390e+01
  4.71888412e+01 -5.94876590e+01 -2.76963443e+01 -2.17248146e+02
 -7.73902111e+01  2.17917389e+02 -3.88887586e+02 -2.51085140e+02
  7.47569318e+02 -7.06372159e+01  3.60672389e+02 -3.78206349e+02]
Intercept: 852230983284981.1


In [49]:
y_hat_train_rate_poly

array([ 1.875, -3.125,  0.625,  1.25 ,  0.625,  1.5  ,  3.   ,  3.5  ,
        0.875,  0.25 , -2.125,  0.375,  0.875, -0.5  ,  1.   ,  1.25 ,
        2.5  , -2.   ,  5.125, -2.125,  4.75 ,  0.625,  1.125,  3.375,
        3.25 ,  0.875,  3.125,  4.125,  3.   ,  2.625,  3.   , -0.625,
       -5.25 , -2.375,  0.25 ,  2.625,  0.875,  1.75 ,  2.375,  0.875,
        2.125,  4.625,  0.375,  2.125,  5.375,  2.875,  1.75 ,  0.875,
        5.875,  1.   ,  2.625,  2.125,  2.5  ,  3.5  ,  1.5  ,  3.25 ,
        3.25 ,  1.875,  2.   ,  1.25 ,  0.5  ,  0.5  ,  3.   ,  3.75 ,
        2.375,  1.375,  1.75 ,  3.125,  4.5  ,  0.625, -1.   ,  4.   ,
        1.5  ,  4.   ,  2.75 , -0.5  ,  1.875,  1.625, -0.625,  1.   ,
        2.75 , -0.25 ,  3.   ,  2.625, -0.375,  1.   ,  1.25 ,  2.75 ,
        2.   ,  1.125,  1.75 ,  2.625, -1.5  ,  2.875,  1.75 ,  2.125,
        1.875,  1.75 ,  4.125,  1.75 ,  1.125,  0.625,  1.75 ,  0.125,
       -8.5  ,  2.625, -4.5  ,  0.75 , -2.   ,  6.25 ,  3.625, -0.75 ,
      

In [48]:
y_train_rate

1      1.5
2      1.5
3      7.5
4      7.5
5      7.5
6      0.5
7      0.5
8      0.5
9      2.5
10     2.5
11     2.5
12    -1.1
13    -1.1
14    -1.1
15     2.4
16     2.4
17     2.4
18    -1.7
19    -1.7
20    -1.7
21     1.1
22     1.1
23     1.1
24     3.5
25     3.5
26     3.5
27     2.4
28     2.4
29     2.4
30     1.8
      ... 
161    0.5
162    3.2
163    3.2
164    3.2
165    3.2
166    3.2
167    3.2
168   -1.1
169   -1.1
170   -1.1
171    5.5
172    5.5
173    5.5
174    5.0
175    5.0
176    5.0
177    2.3
178    2.3
179    2.3
180    3.2
181    3.2
182    3.2
183    3.0
184    3.0
185    3.0
186    1.3
187    1.3
188    1.3
189    0.1
190    0.1
Name: Real_gdp_qtr_growth, Length: 190, dtype: float64